In [1]:
# import libraries
from pylab import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import dates
import pandas as pd
from scipy import stats
import seaborn as sns
from datetime import datetime

# specify number of rows and columns to show
pd.set_option('display.max_rows', 25) 
pd.set_option('display.max_columns', 20)

# set default number format to 3 decimal places
pd.options.display.float_format = '{:40,.3f}'.format

# set ggplot style for plots
plt.style.use('ggplot') 

# Set seaborn style
sns.set(style="whitegrid")
%matplotlib inline

In [2]:
# Import the cleaned dataset

fulldata1 = pd.read_csv('1EveryoneActive_CLEANED.csv')
fulldata2 = pd.read_csv('2EveryoneActive_CLEANED.csv')

# Combine the two imported sets
frames = [fulldata1, fulldata2]
fulldata = pd.concat(frames)

# Drop unused columns
columns = ['ActivityType', 'CitySaveActiveWestminster', 'FirstVisit']
fulldata.drop(columns, inplace=True, axis=1)

# Drop duplicate MemberIDs so we are only counting each member once

# First break off the '0' MembershipIDs since they are unique and we will keep them
fulldata_0 = fulldata.loc[fulldata['MembershipID'] == 0]

# Drop duplicates from the rest of the main dataset, keeping just the last instance of MembershipID
fulldata_IDdrop = fulldata.drop_duplicates(['MembershipID'], keep='last')

# Re-combine the two dataframes
frames = [fulldata_0, fulldata_IDdrop]
allSC = pd.concat(frames)

# Drop all unknown (XXXX) postcodes for spatial analysis work
SC_allpost = allSC.loc[allSC['Postcode'] != 'XXXX']

# SPLIT THE DATA BY SPORTSCENTRE
porch = SC_allpost.loc[SC_allpost['SportsCentre'] == 'Porchester Hall']
qm = SC_allpost.loc[SC_allpost['SportsCentre'] == 'Queen Mother SC']
marsh = SC_allpost.loc[SC_allpost['SportsCentre'] == 'Marshall Street LC']
sey = SC_allpost.loc[SC_allpost['SportsCentre'] == 'Seymour LC']
jub = SC_allpost.loc[SC_allpost['SportsCentre'] == 'Jubilee SC']
lv = SC_allpost.loc[SC_allpost['SportsCentre'] == 'Little Venice SC']
mob = SC_allpost.loc[SC_allpost['SportsCentre'] == 'Moberly SC']
pad = SC_allpost.loc[SC_allpost['SportsCentre'] == 'Paddington Rec Ground']

In [3]:
# Generate counts for each Postcode
post_group = SC_allpost.groupby(['Postcode']).count().reset_index()

# Create new column of the count per Postcode
post_group['PostCount'] = post_group[['MembershipID']]
post_group = post_group[['Postcode','PostCount']]

# Truncate the Postcodes by removing the space in-between them
post_group['Postcode'] = post_group['Postcode'].str.replace(' ', '')

In [4]:
## CALCULATE THE GEODEMOGRAPHIC OF EACH OA BY AGGREGATING UP

# Get the OA Acorn geodem based on which postcode geodem had the highest user count

# Import postcode dataset with OA assigned
pdata = pd.read_csv('Westminster_Postcodes w OA.csv')


In [5]:
# Gets the sums of each postcode by OA.
category_sum = pdata.groupby(['OA11CD','Category', 'CategoryTy', 'CategoryGr'],as_index=False).agg({'PostCount': 'sum'})

# Find the category with the highest count by OA 
high_cat = category_sum.groupby(['OA11CD'])['PostCount'].transform(max) == category_sum['PostCount']
high_cat = category_sum[high_cat]

# Drop the summed amount from the dataframe
high_cat = high_cat.drop(['PostCount'], axis=1)

# Rename the column name for this category the column header
high_cat = high_cat.rename(index=str, columns={"CategoryTy": "CategoryType", 'CategoryGr': 'CategoryGroup'})

#Export this dataframe of Acorn category of SC users aggregated by OA

#high_cat.to_csv('OA_Acorn_SC users.csv', index=False)


In [6]:
# Same as above, but use all Acorn data (not combined Acorn + SC data as above)

pdata2 = pd.read_csv('Acorn geo w OA.csv')

# Create a dataframe of the categories we want. 
pdata2_dropped = pdata2[['CategoryTy','CategoryGr', 'OA11CD', 'Category']]

# The lambda and mode returns either mode if the Category/Category Type/Category Group has multiple modes. Tells us the majority category of each OA
pdata2_cut1 = pdata2_dropped.groupby(['OA11CD'])['Category'].agg(lambda x: pd.Series.mode(x)[0]).to_frame()

pdata2_cut2 = pdata2_dropped.groupby(['OA11CD'])['CategoryTy'].agg(lambda x: pd.Series.mode(x)[0]).to_frame()

pdata2_cut3 = pdata2_dropped.groupby(['OA11CD'])['CategoryGr'].agg(lambda x: pd.Series.mode(x)[0]).to_frame()

# Combine the three dataframes into the single dataframe we'll continue to use. 
pdata_all = pdata2_cut1.merge(pdata2_cut2, how = 'inner', on = 'OA11CD')
pdata_all = pdata_all.merge(pdata2_cut3, how = 'inner', on = 'OA11CD')

pdata_all = pdata_all.rename(index=str, columns={"CategoryTy": "CategoryType", 'CategoryGr': 'CategoryGroup'})
pdata_all = pdata_all.reset_index()

# Export this dataframe of Acorn category users aggregated by OA

#pdata_all.to_csv('OA_Acorn.csv', index=False)


In [7]:
# TIDY DATA FOR SPATIAL INTERACTION MODEL

# Get counts per postcode and sportscentre. 
post_group2 = SC_allpost.groupby(['Postcode', 'SportsCentre']).count().reset_index()
post_group2.sort_values(by=['Postcode'])

# Create new column of the count per Postcode
post_group2['PostCount'] = post_group2[['MembershipID']]
post_group2 = post_group2[['Postcode','SportsCentre','PostCount']]

# Truncate the Postcodes by removing the space in-between them
post_group2['Postcode'] = post_group2['Postcode'].str.replace(' ', '')

# Append the ACORN categories and OA code using Postcode attribute
# Rename OA and ACORN dataframe column name in order to merge
pdata = pdata.rename({'postcode':'Postcode'}, axis='columns')

# User inner merge to drop all non-Westminster postcodes. It only matches ones that we have demographics for
post_merge2 = pd.merge(post_group2, pdata, on='Postcode', how='inner')

# Rename count name to Users
post_merge2 = post_merge2.rename({'PostCount_x':'Users'}, axis='columns')

# Keep only the columns we want
post_merge2 = post_merge2[['Postcode', 'OA11CD','SportsCentre','Users']]

# Gets the user sums by each OA and Sports Centre
user_sum = post_merge2.groupby(['OA11CD','SportsCentre'],as_index=False).agg({'Users': 'sum'})


In [8]:
# Import distances
distances = pd.read_csv('OA_SC distances.csv')

# Melt the distances data into a format we can use, from wide format to long format
d_melt=pd.melt(distances, id_vars=['ID'], value_vars=['Seymour LC', 'Queen Mother SC','Paddington Rec Ground','Moberly SC','Little Venice SC','Jubilee SC','Porchester Hall','Marshall Street LC'])

# Correct column names
d_melt = d_melt.rename({'ID':'OA11CD', 'variable': 'SportsCentre', 'value': 'Distance'}, axis='columns')

# Merge distance data with the above sum data. 
OA_SC_Dist = user_sum.merge(d_melt, on=['OA11CD', 'SportsCentre'])


In [9]:
# Re-attach all other OA details so that we can play with them in the SIM model

#Upload the CSV file with all our variables
SIM_var = pd.read_csv('Westminster OA Variables_SIM.csv')

# Merge the variable data with the above sum data. 
OA_SC_Dist_Var = OA_SC_Dist.merge(SIM_var, on=['OA11CD'])

# Export the melted, tidy data as csv
#OA_SC_Dist.to_csv('Spatial Interaction Data.csv', index=False)

# Export the melted, tidy data along with OA variables as csv
#OA_SC_Dist_Var.to_csv('Spatial Interaction Data_v2.csv', index=False)